In [1]:
from skimage.feature import local_binary_pattern
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.exposure import histogram
import os
import random
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
import time
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
trainingset=[]
histSet=[]
trainingsetHist=[]
testsetHist=[]
testset=np.array([])
labels=np.array([])
time_file = open("time.txt", 'a')
result_file = open("results.txt", 'a')

In [3]:
def describe(images,label,isTrain=0,numPoints=11,radius=10, eps=1e-7):
    global trainingsetHist
    global testsetHist
    for image in images:
        lbp = local_binary_pattern(image, numPoints, radius, method= "uniform" ) 
        (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3),range=(0, numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        if isTrain==0:
            trainingsetHist.append([hist,label])
        else:
            testsetHist.append(hist)
    return

In [4]:
def cutLines(gray):
    edges = cv2.Canny(gray,90,100,apertureSize = 3)
    minLineLength=55
    lines = cv2.HoughLinesP(image=edges,rho=1,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=minLineLength,maxLineGap=0)
 
    linesNew=[ line[0][1] for line in lines if line[0][1]==line[0][3]]
    linesNew=sorted(linesNew)
    finalList=[linesNew[0]]
    for i in range(1,len(linesNew)):
        if linesNew[i]-linesNew[i-1]>=100:
            finalList.append(linesNew[i])
    if len(finalList)>=3 and finalList[1]>=300 and finalList[1]<=800 and finalList[2]>=2000 and finalList[2]<=3000 :
        return gray[finalList[1]:finalList[2],:]
    else:
        return gray[700:2200,:]

In [5]:
def lines_segments(gray):
    th, im_th_otsu = cv2.threshold(gray, 128, 192, cv2.THRESH_OTSU)
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((10,200), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    images = [im_th_otsu[y:y+h, x:x+w] for x,y,w,h in [cv2.boundingRect(ctr) for ctr in ctrs] if(len(sum(im_th_otsu[y:y+h, x:x+w]))>200 and h>40)]
    return images

In [6]:
def run(pathToData='new data'):
    global trainingset
    global labels
    global histSet
    global trainingsetHist
    global testsetHist
    global time_file
    global result_file
        
    data= sorted(os.listdir(pathToData),key=int)

    path=pathToData
    testset=None
    for datafolder in data:
        path+='/'+datafolder
        imageFolders=os.listdir(path)
        for imageFolder in sorted(imageFolders):
            if(imageFolder=='test.png'):
                testset=cv2.imread(path+'/'+imageFolder,cv2.IMREAD_GRAYSCALE)
            else:
                trainingset.append([cv2.imread(path+'/'+imageFolder+'/1.png',cv2.IMREAD_GRAYSCALE),int(imageFolder)])
                trainingset.append([cv2.imread(path+'/'+imageFolder+'/2.png',cv2.IMREAD_GRAYSCALE),int(imageFolder)])
        path=pathToData
        start = time.time()
        for image in trainingset:
            newImage=cutLines(image[0])
            segments=lines_segments(newImage)
            describe(segments,image[1])
        random.shuffle(trainingsetHist)
        for datatrain in trainingsetHist:
            histSet.append(datatrain[0])
            labels=np.append(labels,datatrain[1])

        #------------models-----------------------
        #model = SVC(C=5.0, gamma='auto', probability=True)
        #model= AdaBoostClassifier(n_estimators=250, random_state=0)
        #model=GaussianNB()
        #model=MultinomialNB()
        #model= KNeighborsClassifier(n_neighbors=1)
        model=RandomForestClassifier(max_depth=100,n_estimators=100, random_state=0)
        #model = BaggingClassifier(n_estimators=400,random_state=0)
        #model=MLPClassifier(solver='adam',hidden_layer_sizes=(64, 8), random_state=0,max_iter=20000,learning_rate_init=0.0001)
        #model= GradientBoostingClassifier(n_estimators=100, learning_rate=0.00001,max_depth=50, random_state=0)
        #------------------------------------------
        model.fit(histSet, labels)
        newImage=cutLines(testset)
        segments=lines_segments(newImage)
        describe(segments,-1,1)
        p = model.predict_proba(testsetHist)
        p = np.sum(p, axis=0)
        
        y_pred = model.classes_[np.argmax(p)]
        end = time.time()
        result_file.write(str(int(y_pred)) +'\n')
        t = (end - start)
        time_file.write(str(round(t,2)) + '\n')
        

        trainingset=[]
        trainingsetHist=[]
        labels=np.array([])
        histSet=[]
        testsetHist=[]

In [7]:
def main():
    global time_file
    global result_file
    path=str(input('please enter the path to the dataset folder '))
    run(pathToData=path)
    result_file.close()
    time_file.close()

In [9]:
main()